In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib import colors as mplcolors
import pygplates
import pygmt

from gprm import ReconstructionModel
from gprm.datasets import Rocks, Reconstructions, Paleogeography, Geology
from gprm.utils.raster import to_anchor_plate

import sys
#sys.path.append('/Users/simon/OneDrive/Andes_works//python/')
sys.path.append('../../andes_paper/python/')
import joyful_geochemistry as joy
import joyful_mapping as joymap

import collections

%matplotlib inline
%load_ext autoreload
%autoreload 2


############## Settings for Scotese Paleomap
PaleomapDictionary = {}
PaleomapDictionary['name'] = 'Paleomap'
PaleomapDictionary['reconstruction_model'] = Reconstructions.fetch_Scotese()
PaleomapDictionary['raster_sequence'] = Paleogeography.fetch_Paleomap(resolution='06m')
PaleomapDictionary['maximum_time'] = 540.
PaleomapDictionary['time_bin_size'] = 20.
PaleomapDictionary['anchor_plate_id'] = 0
PaleomapDictionary['raster_anchor_plate_id'] = 0

########## Geochemistry Inputs
#df = joy.geochem_from_csv('../datafiles/geochem_merge_20221026.csv')
#df = joy.geochem_from_csv('../datafiles/geochem_global_20230124.csv',
#                          longitude_field_name='longitude', latitude_field_name='latitude')

model_dir = '../../andes_paper/luffi/REM_surfaces_csv/'
gc_interpolator_dict = joy.make_gc_interpolator_dict(model_dir)



In [ ]:
df = pd.read_csv('../datafiles/geochem_global_20230124_subduction_M2021.csv')
df = gpd.GeoDataFrame(df, geometry=gpd.GeoSeries.from_wkt(df['geometry']), crs='EPSG:4326')

sz_distance_limit = 750.

df = df.query('distance_to_sz<@sz_distance_limit').reset_index(drop=True)

df.plot()


In [ ]:
## MODEL = PaleomapDictionary
time_bin_size = 20
space_bin_size = 2.

calibration = 'luffi'
mohometer_selection = 41

anchor_plate_id = MODEL['anchor_plate_id']
raster_anchor_plate_id = MODEL['raster_anchor_plate_id']

reconstruction_model = MODEL['reconstruction_model']
raster_sequence = MODEL['raster_sequence']

if isinstance(mohometer_selection, list):
    mohometer_description_string = '|'.join(mohometer_selection)
else:
    mohometer_description_string = str(mohometer_selection)



orogen_names = [
    'Cordilleran', 
    'Terra Australis', 
    'Solonker', 
    'Indosinian', 
    'Variscan-Hercynian',
    #'Scotian',
    #'Uralian',
    'Mongol',
    'Kazakh',
    'Caledonia',
    'Alpine-Himalayan'
]


reconstruction_times = np.arange(0,MODEL['maximum_time']+time_bin_size,time_bin_size)

for orogen_name in orogen_names:

    df_sel = joymap.select_orogens(
        df, 
        gdf=None, 
        orogen_names=[orogen_name], 
    )

    elevation_violins = []
    elevation_binned_violins = []
    DEM_violins = []
    residual_violins = []
    residual_binned_violins = []

    for reconstruction_time in reconstruction_times:

        df_filt_r = joymap.geochem_timeslice(df_sel, reconstruction_time, time_bin_size, calibration=calibration,
                                             reconstruction_model=reconstruction_model, anchor_plate_id=anchor_plate_id)

        if df_filt_r is None:
            elevation_violins.append([np.nan, np.nan])
            elevation_binned_violins.append([np.nan, np.nan])
            DEM_violins.append([np.nan, np.nan])
            residual_violins.append([np.nan, np.nan])
            residual_binned_violins.append([np.nan, np.nan])

        else:
            elevation_estimates = joy.get_elevations(df_filt_r, 
                                                     gc_interpolator_dict=gc_interpolator_dict,
                                                     calibration=calibration,
                                                     mohometer_selection=mohometer_selection)

            binned_df = joy.bin_elevations(df_filt_r.geometry, elevation_estimates, space_bin_size)

            DEM = pygmt.grdtrack(points=pd.DataFrame(data={'x':df_filt_r.geometry.x,'y':df_filt_r.geometry.y}),
                                 grid=raster_sequence[reconstruction_time], 
                                 newcolname='z', 
                                 no_skip=True)['z']
            elevations_residuals = elevation_estimates.sub(DEM, axis=0)

            residuals_bdf = joy.bin_elevations(df_filt_r.geometry, elevations_residuals, bin_size_degrees=space_bin_size)

            elevation_violins.append(np.array(elevation_estimates.median().dropna()))
            elevation_binned_violins.append(np.array(binned_df.median_elevation.dropna()))
            
            DEM_violins.append(np.array(DEM.dropna()))
            
            residual_violins.append(np.array(elevations_residuals.median().dropna()))
            residual_binned_violins.append(np.array(residuals_bdf.median_elevation.dropna()))
        #break


    fig = plt.figure(figsize=(7,7))
    gs = fig.add_gridspec(nrows=2, ncols=1, wspace=0.1, hspace=0., height_ratios=[1,1])

    ax = fig.add_subplot(gs[1, 0])
    
    color='black'
    tmp = pd.DataFrame(elevation_violins)
    plt.fill_between(reconstruction_times, 
                     tmp.quantile(q=0.25,axis=1), tmp.quantile(q=0.75,axis=1),
                     color=color, alpha=0.2, edgecolor=None)
    parts = ax.boxplot(elevation_violins, 
                       positions=reconstruction_times-time_bin_size*0.2, 
                       widths=time_bin_size*0.4, 
                       patch_artist=True, manage_ticks=False, showfliers=False)
    for pc in parts['boxes']:
        pc.set_facecolor(color)
        pc.set_alpha(0.5)
        pc.set_edgecolor(color)
    
        
    color='navy'
    tmp = pd.DataFrame(DEM_violins)
    plt.fill_between(reconstruction_times, 
                     tmp.quantile(q=0.25,axis=1), tmp.quantile(q=0.75,axis=1),
                     color=color, alpha=0.2, linewidth=0)
    parts = ax.boxplot(DEM_violins, 
                       positions=reconstruction_times+time_bin_size*0.2, 
                       widths=time_bin_size*0.4, 
                       patch_artist=True, manage_ticks=False, showfliers=False)
    for pc in parts['boxes']:
        pc.set_facecolor(color)
        pc.set_alpha(0.5)
        pc.set_edgecolor(color)
        
    ax.set_title(orogen_name)
    ax.set_xlim(0,540)
    ax.set_ylim(-1000,5500)
    ax.set_xlabel('Reconstruction Age [Ma]')
    ax.set_ylabel('Elevation [m]')

    ax = fig.add_subplot(gs[0, 0])
    color='black'
    parts = ax.boxplot(residual_violins, 
                       positions=reconstruction_times, 
                       widths=time_bin_size*0.8, 
                       patch_artist=True, manage_ticks=False, showfliers=False)        
    cmap = plt.cm.ScalarMappable(cmap='seismic', norm=mplcolors.Normalize(-4000,4000))
    test_mean = [np.mean(x) for x in residual_violins]
    for pc, boxcolor in zip(parts['boxes'], cmap.to_rgba(test_mean)):
        pc.set_facecolor(boxcolor)
        pc.set_alpha(0.5)
        pc.set_edgecolor(color)
        
    ax.set_xlim(0,540)
    ax.set_ylim(-1500,5000)
    ax.set_xticks([])
    ax.set_ylabel('Elevation Residual [m]')
    
    fig.savefig(
        '../images/boxplots/paleomap_comparison_by_orogen_{:s}_{:s}_{:s}.png'.format(
            orogen_name, 
            calibration, 
            mohometer_description_string), 
        dpi=600)

    #plt.show()
    plt.close()
    #break
